# Multi-model Aggregation

> Geographical Hierarchical Forecasting on Australian Tourism Data using multiple models for each level in the hierarchy.

This notebook extends the classic Australian Domestic Tourism (`Tourism`) geographical aggregation example to showcase how `HierarchicalForecast` can be used to produce coherent forecasts when **different forecasting models are applied at each level of the hierarchy**. We will use the `Tourism` dataset, which contains monthly time series of the number of visitors to each state of Australia.

Specifically, we will demonstrate fitting a diverse set of models across the hierarchical levels. This includes statistical models like `AutoETS` from `StatsForecast`, machine learning models such as `HistGradientBoostingRegressor` using `MLForecast`, and neural network models like `NBEATS` from `NeuralForecast`. After generating these base forecasts, we will reconcile them using `BottomUp` reconciliation from `HierarchicalForecast`.


You can run these experiments using CPU or GPU with Google Colab.

<a href='https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism-Multimodel.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast mlforecast datasetsforecast sklearn neuralforecast

## 1. Load and Process Data

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset from the [Forecasting: Principles and Practice](https://otexts.com/fpp3/) book.

The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
Y_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
Y_df = Y_df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
Y_df.insert(0, 'Country', 'Australia')
Y_df = Y_df[['Country', 'Region', 'State', 'ds', 'y']]
Y_df['ds'] = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(Y_df['ds'], freq='Q').to_timestamp()
Y_df_first = Y_df.groupby(['Country', 'Region', 'State', 'ds'], as_index=False).agg({'y':'sum'})
Y_df_first.head()

,Country,Region,State,ds,y
0,Australia,Adelaide,South Australia,1998-01-01,658.553895
1,Australia,Adelaide,South Australia,1998-04-01,449.853935
2,Australia,Adelaide,South Australia,1998-07-01,592.904597
3,Australia,Adelaide,South Australia,1998-10-01,524.242760
4,Australia,Adelaide,South Australia,1999-01-01,548.394105


The dataset can be grouped in the following hierarchical structure.

In [ ]:
spec = [
    ['Country'],
    ['Country', 'State'],
    ['Country', 'State', 'Region']
]

Using the `aggregate` function from `HierarchicalForecast` we can get the full set of time series.

In [ ]:
%%capture
from hierarchicalforecast.utils import aggregate

In [ ]:
Y_df, S_df, tags = aggregate(Y_df_first, spec)

### Split Train/Test sets

We use the final two years (8 quarters) as test set.

In [ ]:
Y_test_df = Y_df.groupby('unique_id', as_index=False).tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

## 2. Computing different models for different hierarchies

In this section, we illustrate how to fit a different type of model for each level of the hierarchy. In particular, for each level, we will fit the following models:

* **Country**: `AutoETS` model from `StatsForecast`.
* **Country/State**: `HistGradientBoostingRegressor` model from `scikit-learn` through the `MLForecast` API.
* **Country/State/Region**: `NBEATS` model from `NeuralForecast`.

In [ ]:
%%capture

from statsforecast.core import StatsForecast
from statsforecast.models import AutoETS

from mlforecast import MLForecast
from sklearn.ensemble import HistGradientBoostingRegressor

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS

# The following helps suppress the Pytorch logging information
import logging  
logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)  
logging.getLogger('lightning_fabric').setLevel(logging.ERROR)

This `fit_predict_any_models` function provides a unified interface for training and forecasting with models from `StatsForecast`, `MLForecast`, and `NeuralForecast`. By abstracting away the specific fit and predict methods of each library, it simplifies the process of applying diverse model types across different levels of a hierarchy. This utility streamlines the experimentation and implementation of multi-model hierarchical forecasting strategies, enhancing code readability and reusability.

In [ ]:
def fit_predict_any_models(models: StatsForecast | MLForecast | NeuralForecast, df: pd.DataFrame, h:int):
    if isinstance(models, StatsForecast):
        yhat = models.forecast(df=df, h=h, fitted=True)
        yfitted = models.forecast_fitted_values()

    elif isinstance(models, MLForecast):
        models.fit(df, fitted=True)
        yhat = models.predict(new_df=df, h=h)
        yfitted = models.forecast_fitted_values()

    elif isinstance(models, NeuralForecast):
        models.fit(df=df, val_size=h)
        yhat = models.predict()
        yfitted = models.predict_insample(step_size=h)

    else:
        print('Model is not a StatsForecast, MLForecast or NeuralForecast object.')

    return yhat, yfitted

In [ ]:
%%capture

stat_models = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], freq='QS', n_jobs=-1)
ml_models = MLForecast(models = {'gbm': HistGradientBoostingRegressor()}, freq='QS', lags=[1, 4])
neural_models = NeuralForecast(models=[NBEATS(h=8, input_size=16, learning_rate=1e-3, enable_progress_bar=False, logger=False, enable_model_summary=False)],freq='QS')

In [ ]:
models = {
    'Country': stat_models,
    'Country/State': ml_models,
    'Country/State/Region': neural_models
}

In our case we only have one model per type of `Forecast` however it may be useful to determine which one is the best and which one we'll be using in each level, i.e:

In [ ]:
best_models = {
    'Country': 'AutoETS',
    'Country/State': 'gbm',
    'Country/State/Region': 'NBEATS'
}

In [ ]:
%%capture

Y_hat = []
Y_fitted = []

for key, value in tags.items():
    df_level = Y_train_df.query('unique_id.isin(@value)')
    yhat_level, yfitted_level = fit_predict_any_models(models[key], df_level, h=8)
    
    yhat_level = yhat_level[['unique_id', 'ds', best_models[key]]].rename(columns={best_models[key]: 'best_pred'})
    yfitted_level = yfitted_level[['unique_id', 'ds', 'y', best_models[key]]].rename(columns={best_models[key]: 'best_pred'})

    Y_hat.append(yhat_level)
    Y_fitted.append(yfitted_level)

Y_hat_df = pd.concat(Y_hat, ignore_index=True)
Y_fitted_df = pd.concat(Y_fitted, ignore_index=True)

## 3. Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. In this example we use `BottomUp` reconciler.

In [ ]:
from hierarchicalforecast.methods import BottomUp, MinTrace, TopDown
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    TopDown(method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df, tags=tags)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [ ]:
Y_rec_df.head()

,unique_id,ds,best_pred,best_pred/BottomUp,best_pred/MinTrace_method-mint_shrink,best_pred/TopDown_method-forecast_proportions
0,Australia,2016-01-01,25990.068004,24776.339488,26205.549950,25990.068004
1,Australia,2016-04-01,24458.490282,22953.216803,24834.758654,24458.490282
2,Australia,2016-07-01,23974.055984,22466.372158,25318.922379,23974.055984
3,Australia,2016-10-01,24563.454495,23772.236997,26448.698961,24563.454495
4,Australia,2017-01-01,25990.068004,25599.068889,26806.016540,25990.068004


## 4. Evaluation 

The `HierarchicalForecast` package includes an `evaluate` function to evaluate the different hierarchies. To evaluate models we use `mase` metric and compare it to base predictions.

In [ ]:
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import mase
from functools import partial

In [ ]:
eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']

df = Y_rec_df.merge(Y_test_df, on=['unique_id', 'ds'])
evaluation = evaluate(df = df,
                      tags = eval_tags,
                      train_df = Y_train_df,
                      metrics = [partial(mase, seasonality=4)])

evaluation.columns = ['level', 'metric', 'Base', 'BottomUp', 'MinTrace(mint_shrink)', 'TopDown(forecast_proportions)']
numeric_cols = evaluation.select_dtypes(include="number").columns
evaluation[numeric_cols] = evaluation[numeric_cols].map('{:.2f}'.format).astype(np.float64)

In [ ]:
evaluation.query('metric == "mase"')

,level,metric,Base,BottomUp,MinTrace(mint_shrink),TopDown(forecast_proportions)
0,Total,mase,1.59,2.94,0.55,1.59
1,State,mase,2.17,1.88,1.86,2.36
2,Regions,mase,1.34,1.34,1.50,1.49
3,Overall,mase,1.42,1.41,1.53,1.57
